# preprocess1
>제공데이터 형식에 맞게 수집데이터(raw)를 형태 변환  

수집 데이터에 따른 전처리 과정 분류 (변수명 기준)
  - 1. APT매매가격지수
  - 2. 업무용_실거래가_연면적당평균(만원/㎡)
  - 3. 기준금리(%)
  - 4. 소비자물가지수
  - 5. 건설비공사지수  
  : 건설_소계, 주거용건물, 비주거용건물, 건축보수
  - 6. 표준지공사비지수평균(원/㎡)
  - 7. 상업용 임대동향 지표(오피스_)  
  : 평균연면적(㎡),공실률(%),전환율(%),임대료(천원/㎡), 임대수입(%),기타수입(%),영업경비(%),  
  순영업소득(%),순영업소득(천원/㎡), 소득수익률(%),자본수익률(%),투자수익률(%)
  - 8. 주거용_실거래가_건물면적당평균(원/㎡)  
  : 주거용, 단독다가구, 연립다세대, 아파트, 오피스텔
  - 9. 오피스매매가격지수
  - 10. 권역

> 제공데이터 변수(15) + 수집데이터 변수(28)   

> processed1.csv : 총 43개 변수

In [1]:
## 라이브러리 호출
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from pickle import TRUE

# 한글폰트 설정
from matplotlib import rc
rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False #음수에 (-)표현
pd.set_option('display.max_columns',None)

# Warning 방지
import warnings
warnings.filterwarnings('ignore')

## 0. 제공 데이터
- 학습데이터셋(2014~23), 평가데이터셋(2024) 결합  
- 한글 변환 : 기존 제공데이터의 변수명을 분석에 용이하도록 한글로 변환  
- '동' : 공백을 모두 제거
   - '문정동', '문정동  '과 같이 공백이 존재하는 경우가 있음
   - 모든 '동'의 공백을 제거하여 동명 통일

학습(2014-2023) + 평가(2024) 데이터셋 결합

In [2]:
# 제공데이터 : 학습(2014~23), 평가데이터셋(2024) 결합
df0_train = pd.read_excel("../data/raw/240904_오피스 매매데이터_빅콘테스트(부동산빅데이터플랫폼).xlsx")
df0_test = pd.read_excel("../data/raw/240912_오피스 매매데이터_빅콘테스트 제출용_2024년거래만.xlsx")
df0 = pd.concat([df0_train, df0_test], axis=0)

df0.shape  #972개의 행, 15개의 열로 구성

(972, 15)

변수명 한글 변환

In [3]:
# 변수명 한글 변환
column_mapping = {
    'bld_type': '건물유형',
    'add_si': '시',
    'add_gu': '구',
    'add_dong': '동',
    'gfa_crr': '현재연면적',
    'gfa_dlt': '거래당시연면적',
    'land_area': '거래당시토지면적',
    'floor_gr': '지상층',
    'floor_bm': '지하층',
    'year_con': '준공년도',
    'price_tr': '거래가',
    'class_type': '빌딩비고',
    'class_dlt ': '거래비고',
    'dealt_yr': '거래년도',
    'dealt_qr': '거래분기'
}
df0.rename(columns=column_mapping,inplace=True)

df0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 972 entries, 0 to 27
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   거래년도      972 non-null    int64  
 1   거래분기      972 non-null    object 
 2   시         972 non-null    object 
 3   구         972 non-null    object 
 4   동         972 non-null    object 
 5   거래당시연면적   972 non-null    float64
 6   거래당시토지면적  972 non-null    float64
 7   현재연면적     36 non-null     float64
 8   빌딩비고      125 non-null    object 
 9   거래비고      20 non-null     object 
 10  거래가       972 non-null    int64  
 11  지상층       970 non-null    float64
 12  지하층       970 non-null    float64
 13  준공년도      969 non-null    float64
 14  건물유형      972 non-null    object 
dtypes: float64(6), int64(2), object(7)
memory usage: 121.5+ KB


'동'의 표기방식 통일

In [4]:
# '문정동 '와 같이 공백이 있는 경우가 존재
print("**전처리 전** :\n", df0[df0['동'].str.startswith('문정')]['동'].unique(),'\n')

# 모든 '동'열에 대해서 공백 삭제
df0['동'] = df0['동'].str.replace(' ', '', regex=True)
print("**전처리 후** :\n", df0[df0['동'].str.startswith('문정')]['동'].unique())

**전처리 전** :
 ['문정동' '문정동 '] 

**전처리 후** :
 ['문정동']


In [5]:
# 전처리 완료 제공데이터
display(df0.head())
display(df0.tail())

,거래년도,거래분기,시,구,동,거래당시연면적,거래당시토지면적,현재연면적,빌딩비고,거래비고,거래가,지상층,지하층,준공년도,건물유형
0,2014,1Q,서울,영등포구,여의도동,27202.05,3602.0,NaN,NaN,NaN,14600,12.0,4.0,1985.0,집합
1,2014,1Q,서울,강남구,논현동,3516.43,713.6,NaN,NaN,NaN,8100,7.0,2.0,2004.0,일반
2,2014,1Q,서울,서대문구,충정로3가,25984.49,2705.0,NaN,NaN,NaN,13900,17.0,4.0,1995.0,일반
3,2014,1Q,서울,서대문구,충정로3가,31894.78,3772.0,NaN,NaN,NaN,10500,17.0,4.0,1987.0,집합
4,2014,1Q,서울,용산구,한남동,4370.31,730.5,NaN,NaN,NaN,25200,5.0,4.0,2012.0,일반


,거래년도,거래분기,시,구,동,거래당시연면적,거래당시토지면적,현재연면적,빌딩비고,거래비고,거래가,지상층,지하층,준공년도,건물유형
23,2024,2Q,서울,서초구,서초동,29784.83,4588.60,NaN,NaN,NaN,22500,14.0,5.0,1990.0,일반
24,2024,2Q,서울,구로구,구로동,3556.72,819.70,NaN,NaN,NaN,12100,7.0,2.0,1990.0,일반
25,2024,2Q,서울,영등포구,대림동,5464.08,1216.00,NaN,NaN,NaN,13300,10.0,2.0,1991.0,일반
26,2024,2Q,서울,광진구,자양동,4104.82,1541.40,NaN,NaN,NaN,14900,5.0,1.0,1990.0,일반
27,2024,2Q,서울,동대문구,신설동,7247.65,804.51,NaN,NaN,NaN,19600,14.0,3.0,2008.0,일반


In [6]:
#df0.to_csv('../data/processed_제공데이터.csv',index=False, encoding='cp949')

## 1. APT매매가격지수
- 출처 : [KB부동산 데이터 허브 - 월간 아파트 매매가격지수](https://data.kbland.kr/kbstats/wmh?tIdx=HT01&tsIdx=monthAptSalePriceInx)(아파트 매매 완료)
- raw_data명 : '월간 아파트 매매가격지수_20240929.xlsx'
- raw_data형태 : 월별, 구별, APT매매가격지수
- 형태변환 : 년도, 분기, 구별, APT매매가격지수_평균 계산
- 기간 : 2014.08 - 2024.08, 전국

raw_data

In [7]:
df1 = pd.read_excel('../data/raw/월간 아파트 매매가격지수_20240929.xlsx')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 278 entries, 0 to 277
Columns: 122 entries, 지역명 to 2024-08-01 09:00:52
dtypes: float64(7), object(115)
memory usage: 265.1+ KB


형태 변환

In [8]:
# 서울(index:28까지) + '성남시 분당구' 데이터 추출
df1_seoul = df1[(df1.index < 29) | (df1['지역명'].str.contains('성남시 분당구'))]
df1_seoul['지역명'].tail()

# 구별 형태 변환
df1_melt=pd.melt(df1_seoul,id_vars=['지역명'])

# 날짜형식 변환
df1_melt['variable']=pd.to_datetime(df1_melt['variable'],format='%Y-%m-%d') 

# 연도 추출
df1_melt['dealt_yr']=df1_melt['variable'].dt.year

# 분기별로 변환
conditions = [
    (df1_melt['variable'].dt.month >= 1) & (df1_melt['variable'].dt.month <= 3),
    (df1_melt['variable'].dt.month >= 4) & (df1_melt['variable'].dt.month <= 6),
    (df1_melt['variable'].dt.month >= 7) & (df1_melt['variable'].dt.month <= 9),
    (df1_melt['variable'].dt.month >= 10) & (df1_melt['variable'].dt.month <= 12)
]
choices = ['1Q', '2Q', '3Q', '4Q']
df1_melt['dealt_qr'] = np.select(conditions, choices)

# 변수명 통일 
df1_melt.rename(columns={'value':'APT매매가격지수','지역명':'구', 'dealt_yr': '거래년도','dealt_qr': '거래분기'}, inplace=True)
df1_melt.loc[df1_melt['구'] == '성남시 분당구', '구'] = '분당구'

# 지역 선택 : '전국', '강북14개구', '강남11개구'를 제외
df1_filtered = df1_melt[~df1_melt['구'].isin(['전국', '강북14개구', '강남11개구','서울'])]

# 분기별, 구별 APT매매가격지수 평균 계산
df1_final=df1_filtered.groupby(['구','거래년도','거래분기'])['APT매매가격지수'].mean()
df1_final=df1_final.reset_index()

display(df1_final.head())
display(df1_final.tail())

,구,거래년도,거래분기,APT매매가격지수
0,강남구,2014,3Q,55.878059
1,강남구,2014,4Q,56.321412
2,강남구,2015,1Q,56.618775
3,강남구,2015,2Q,57.522446
4,강남구,2015,3Q,58.733925


,구,거래년도,거래분기,APT매매가격지수
1061,중랑구,2023,3Q,91.183959
1062,중랑구,2023,4Q,90.137804
1063,중랑구,2024,1Q,89.365728
1064,중랑구,2024,2Q,88.01069
1065,중랑구,2024,3Q,88.117279


## 2. 업무용_실거래가_연면적당평균(만원/㎡)
- 출처 : [국토교통부 - 실거래가 공개시스템 - 상업업무용](https://rt.molit.go.kr/pt/xls/xls.do?&mobileAt=)
- 기간 : 2014.01 - 2024.10
- raw_data형식 : 월별, 건물별(시구동 표기), 실거래가, 연면적
- 형태 변환 : 년도, 분기, 동별, 실거래가_연면적당평균(만원/㎡) 계산

### 2-1.성남시 분당
raw_data명 : '상업업무용_실거래가_분당/상업업무용(매매)_실거래가_분당_연도.csv'

In [9]:
directory_path = '../data/raw/상업업무용_실거래가_분당'

# 빈 리스트 생성하여 데이터프레임을 저장
dataframes = []

# 디렉토리 내 모든 CSV 파일 불러오기
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):  # CSV 파일인지 확인
        file_path = os.path.join(directory_path, filename)
        df = pd.read_csv(file_path,encoding='ANSI')  # 파일 읽기
        dataframes.append(df)  # 리스트에 추가

# 모든 분당구 데이터 병합
df2_bundang = pd.concat(dataframes, ignore_index=True)
df2_bundang.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14820 entries, 0 to 14819
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   NO         14820 non-null  int64  
 1   시군구        14820 non-null  object 
 2   유형         14820 non-null  object 
 3   지번         14820 non-null  object 
 4   도로명        14432 non-null  object 
 5   용도지역       14820 non-null  object 
 6   건축물주용도     14820 non-null  object 
 7   도로조건       14820 non-null  object 
 8   전용/연면적(㎡)  14820 non-null  float64
 9   대지면적(㎡)    14820 non-null  object 
 10  거래금액(만원)   14820 non-null  object 
 11  층          7943 non-null   float64
 12  매수         14820 non-null  object 
 13  매도         14820 non-null  object 
 14  계약년월       14820 non-null  int64  
 15  계약일        14820 non-null  int64  
 16  지분구분       1335 non-null   object 
 17  건축년도       14362 non-null  float64
 18  해제사유발생일    14820 non-null  object 
 19  거래유형       14820 non-null  object 
 20  중개사소재지

형태 변환

In [10]:
# 업무용 데이터 추출
df21_office = df2_bundang[df2_bundang['건축물주용도']=='업무']
df21_office = df21_office[['시군구','용도지역','전용/연면적(㎡)','도로조건','대지면적(㎡)','층','거래금액(만원)','계약년월']]

# 시간관련 열 datetime 변환
df21_office['계약년월']=pd.to_datetime(df21_office['계약년월'],format='%Y%m')

# 연도 추출
df21_office['거래년도']=df21_office['계약년월'].dt.year

# 분기 추출
conditions = [
    (df21_office['계약년월'].dt.month >= 1) & (df21_office['계약년월'].dt.month <= 3),
    (df21_office['계약년월'].dt.month >= 4) & (df21_office['계약년월'].dt.month <= 6),
    (df21_office['계약년월'].dt.month >= 7) & (df21_office['계약년월'].dt.month <= 9),
    (df21_office['계약년월'].dt.month >= 10) & (df21_office['계약년월'].dt.month <= 12)
]
choices = ['1Q', '2Q', '3Q', '4Q']
df21_office['거래분기'] = np.select(conditions, choices)

# 시, 구, 동 추출
df21_office[['시', '구', '동']] = df21_office['시군구'].str.extract(r'(\S+시)\s(\S+구)\s(.*)')
df21_office['시']='성남'

# 연면적당_실거래가 계산
df21_office['거래금액(만원)'] = df21_office['거래금액(만원)'].str.replace(',', '').astype('float')
df21_office['연면적당가격(만원/㎡)']=df21_office['거래금액(만원)']/df21_office['전용/연면적(㎡)']

# 업무용_실거래가_연면적당평균 계산
df21_final= df21_office.groupby(['거래년도','거래분기','시', '구', '동'])['연면적당가격(만원/㎡)'].agg(
    업무용_실거래가_연면적당평균='mean'
 ).reset_index()
df21_final.rename(columns={'업무용_실거래가_연면적당평균':'업무용_실거래가_연면적당평균(만원/㎡)'}, inplace=True)

display(df21_final.head())
display(df21_final.tail())


,거래년도,거래분기,시,구,동,업무용_실거래가_연면적당평균(만원/㎡)
0,2014,1Q,성남,분당구,구미동,354.012138
1,2014,1Q,성남,분당구,궁내동,268.600591
2,2014,1Q,성남,분당구,금곡동,268.101328
3,2014,1Q,성남,분당구,삼평동,436.963754
4,2014,1Q,성남,분당구,서현동,610.069339


,거래년도,거래분기,시,구,동,업무용_실거래가_연면적당평균(만원/㎡)
295,2024,3Q,성남,분당구,삼평동,1684.245351
296,2024,3Q,성남,분당구,서현동,368.107851
297,2024,3Q,성남,분당구,수내동,686.060872
298,2024,3Q,성남,분당구,야탑동,361.689815
299,2024,3Q,성남,분당구,정자동,737.383506


### 2-2.서울
raw_data명 : '상업업무용_실거래가_서울/상업업무용(매매)_실거래가_연도.csv'

In [11]:
directory_path = '../data/raw/상업업무용_실거래가_서울'

# 빈 리스트 생성하여 데이터프레임을 저장
dataframes = []

# 디렉토리 내 모든 CSV 파일 불러오기
for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):  # CSV 파일인지 확인
        file_path = os.path.join(directory_path, filename)
        df =  pd.read_csv(file_path, encoding='ANSI')  # 파일 읽기
        dataframes.append(df)  # 리스트에 추가

# 모든 서울 데이터 병합
df2_seoul = pd.concat(dataframes, ignore_index=True)

형태 변환

In [12]:
# 업무용 데이터만 추출
df22_office = df2_seoul[df2_seoul['건축물주용도']=='업무']
df22_office = df22_office[['시군구','용도지역','전용/연면적(㎡)','도로조건','대지면적(㎡)','층','거래금액(만원)','계약년월']]

# 시간관련 열 datetime 변환
df22_office['계약년월']=pd.to_datetime(df22_office['계약년월'],format='%Y%m')

# 연도 추출
df22_office['거래년도']=df22_office['계약년월'].dt.year

# 분기 추출
conditions = [
    (df22_office['계약년월'].dt.month >= 1) & (df22_office['계약년월'].dt.month <= 3),
    (df22_office['계약년월'].dt.month >= 4) & (df22_office['계약년월'].dt.month <= 6),
    (df22_office['계약년월'].dt.month >= 7) & (df22_office['계약년월'].dt.month <= 9),
    (df22_office['계약년월'].dt.month >= 10) & (df22_office['계약년월'].dt.month <= 12)
]
choices = ['1Q', '2Q', '3Q', '4Q']
df22_office['거래분기'] = np.select(conditions, choices)

# 시,구,동 추출
df22_office[['시', '구', '동']] = df22_office['시군구'].str.extract(r'(\S+시)\s(\S+구)\s(.*)')
df22_office['시']='서울'

# 연면적당 실거래가 계산
df22_office['거래금액(만원)'] = df22_office['거래금액(만원)'].str.replace(',', '').astype('float')
df22_office['연면적당가격(만원/㎡)']=df22_office['거래금액(만원)']/df22_office['전용/연면적(㎡)']

# 업무용_실거래가_연면적당평균 계산
df22_final = df22_office.groupby(['거래년도','거래분기','시', '구', '동'])['연면적당가격(만원/㎡)'].agg(
    업무용_실거래가_연면적당평균='mean'
 ).reset_index()
df22_final.rename(columns={'업무용_실거래가_연면적당평균':'업무용_실거래가_연면적당평균(만원/㎡)'}, inplace=True)

display(df22_final.head())
display(df22_final.tail())

,거래년도,거래분기,시,구,동,업무용_실거래가_연면적당평균(만원/㎡)
0,2014,1Q,서울,강남구,대치동,365.933209
1,2014,1Q,서울,강남구,삼성동,1027.440970
2,2014,1Q,서울,강남구,수서동,480.404551
3,2014,1Q,서울,강남구,역삼동,428.642234
4,2014,1Q,서울,강남구,청담동,518.225736


,거래년도,거래분기,시,구,동,업무용_실거래가_연면적당평균(만원/㎡)
3744,2024,3Q,서울,중구,장교동,955.953387
3745,2024,3Q,서울,중구,저동2가,728.675525
3746,2024,3Q,서울,중구,흥인동,488.021295
3747,2024,3Q,서울,중랑구,상봉동,364.758698
3748,2024,4Q,서울,영등포구,영등포동4가,603.106791


### 서울+분당

In [13]:
df2_final = pd.concat([df21_final, df22_final], axis=0).reset_index(drop=True)

display(df2_final.head())
display(df2_final.tail())

,거래년도,거래분기,시,구,동,업무용_실거래가_연면적당평균(만원/㎡)
0,2014,1Q,성남,분당구,구미동,354.012138
1,2014,1Q,성남,분당구,궁내동,268.600591
2,2014,1Q,성남,분당구,금곡동,268.101328
3,2014,1Q,성남,분당구,삼평동,436.963754
4,2014,1Q,성남,분당구,서현동,610.069339


,거래년도,거래분기,시,구,동,업무용_실거래가_연면적당평균(만원/㎡)
4044,2024,3Q,서울,중구,장교동,955.953387
4045,2024,3Q,서울,중구,저동2가,728.675525
4046,2024,3Q,서울,중구,흥인동,488.021295
4047,2024,3Q,서울,중랑구,상봉동,364.758698
4048,2024,4Q,서울,영등포구,영등포동4가,603.106791


In [14]:
df2_final.to_csv('../data/processed1_업무용_실거래가_연면적당평균.csv', index=False, encoding='cp949')

## 3. 기준금리(%)
- 출처 : [e-나라지표 - 금리](https://www.index.go.kr/unity/potal/main/EachDtlPageDetail.do?idx_cd=1073)
- raw_data명 : '107301_20240930121303811_excel.xlsx'
- raw_data형식 : 연도분기별, 기준금리
- 기간 : 2014.1Q - 2024.2Q
- 형태변환 : 연도별, 분기별, 기준금리(%)

raw_data

In [15]:
intrate = pd.read_excel('../data/raw/107301_20240930121303811_excel.xlsx')
intrate.iloc[2,0] = '분기' # 시간 관련 데이터만 추출
intrate = intrate.rename(columns=intrate.iloc[2])
intrate = intrate.iloc[3:45]
intrate.head()

,분기,국고채 3년(평균),국고채 5년(평균),국고채 10년(평균),"회사채 3년(AA-, 평균)",CD 91물(평균),"콜금리(1일물,평균)",기준금리
3,20141/4,2.87,3.20,3.57,3.30,2.65,2.49,2.50
4,20142/4,2.82,3.07,3.41,3.25,2.65,2.49,2.50
5,20143/4,2.50,2.73,3.05,2.89,2.51,2.35,2.00
6,20144/4,2.17,2.36,2.71,2.51,2.16,2.01,2.00
7,20151/4,1.98,2.08,2.35,2.26,2.07,1.93,1.75


형태 변환

In [16]:
# 연도 추출
intrate['거래년도'] = intrate['분기'].str[:4].astype(int)
# 분기 추출
intrate['거래분기'] = intrate['분기'].str[4] + 'Q'
intrate.drop(["분기"], axis=1, inplace=True)
intrate = intrate.iloc[:,-3:]

# 변수명 변환
df3_final = intrate.rename(columns={'기준금리':'기준금리(%)'}).reset_index(drop=True)

display(df3_final.head())
display(df3_final.tail())

,기준금리(%),거래년도,거래분기
0,2.50,2014,1Q
1,2.50,2014,2Q
2,2.00,2014,3Q
3,2.00,2014,4Q
4,1.75,2015,1Q


,기준금리(%),거래년도,거래분기
37,3.50,2023,2Q
38,3.50,2023,3Q
39,3.50,2023,4Q
40,3.50,2024,1Q
41,3.50,2024,2Q


## 4. 소비자물가지수
- 출처 : [지표누리(정부) - 소비자물가지수](https://www.index.go.kr/unify/idx-info.do?idxCd=4226)
- raw_data명 : '소비자물가지수_2020100__20240930125319.xlsx'
- raw_data형식 : 시도별, 연도분기별, 소비자물가지수
- 기간 : 2014.1Q - 2024.2Q, 서울시+성남시 데이터
- 형태변환 : 연도별, 분기별, 시도별, 소비자물가지수

raw_data

In [17]:
# 데이터 불러오기
con = pd.read_excel('../data/raw/소비자물가지수_2020100__20240930125319.xlsx')
con.head()

,시도별,시점,데이터
0,서울특별시,2014.1/4,92.885
1,NaN,2014.2/4,93.334
2,NaN,2014.3/4,93.638
3,NaN,2014.4/4,93.343
4,NaN,2015.1/4,94.005


형태 변환

In [18]:
# '시'명 입력력
con.loc[:41,'시도별'] = '서울'
con.loc[42:,'시도별'] = '성남'
con.head()

# 연도 추출
con['거래년도'] = con['시점'].str[:4].astype(int)
# 분기 추출
con['거래분기'] = con['시점'].str[5] + 'Q'
con.drop(["시점"], axis=1, inplace=True)

# 변수명 통일
df4_final = con.rename(columns={'시도별':'시','데이터':'소비자물가지수'})

display(df4_final.head())
display(df4_final.tail())

,시,소비자물가지수,거래년도,거래분기
0,서울,92.885,2014,1Q
1,서울,93.334,2014,2Q
2,서울,93.638,2014,3Q
3,서울,93.343,2014,4Q
4,서울,94.005,2015,1Q


,시,소비자물가지수,거래년도,거래분기
79,성남,111.02,2023,2Q
80,성남,112.09,2023,3Q
81,성남,112.75,2023,4Q
82,성남,113.53,2024,1Q
83,성남,113.89,2024,2Q


## 5. 건설비공사지수
건설_소계, 주거용건물, 비주거용건물, 건축보수
- 출처 : [통계청 - 건설공사비지수](https://kosis.kr/statHtml/statHtml.do?orgId=397&tblId=DT_39701_A003&vw_cd=MT_ZTITLE&list_id=P2_1&scrId=&seqNo=&lang_mode=ko&obj_var_id=&itm_id=&conn_path=MT_ZTITLE&path=%252FstatisticsList%252FstatisticsListIndex.do)
- raw_data명 : '건설공사비지수_2020년기준__20241019205225.xlsx'
- raw_data형식 : 연도분기별, 건설공사비지수:건설_소계, 주거용건물, 비주거용건물, 건축보수
- 기간 : 2014.01 - 2024.06
- 형태변환 : 연도별, 분기별, 건설공사비지수 평균:건설_소계, 주거용건물, 비주거용건물, 건축보수

 raw_data

In [19]:
# 데이터 불러오기
coc = pd.read_excel('../data/raw/건설공사비지수_2020년기준__20241019205225.xlsx')
coc = coc.rename(columns=coc.iloc[0])
coc = coc.rename(columns={'소계':'건설_소계'})
coc = coc.iloc[2:,]
coc. tail()

,시점,건설_소계,주거용건물,비주거용건물,건축보수
123,2024.02,130.05,129.21,129.77,128.64
124,2024.03,130.05,129.14,129.74,128.63
125,2024.04,130.08,129.1,129.71,128.64
126,2024.05,130.2,129.08,129.74,128.73
127,2024.06,130.11,128.95,129.62,128.73


형태 변환

In [20]:
# 연도 추출
coc['거래년도'] = coc['시점'].str[:4].astype(int)

# 분기 추출
coc['월'] = coc['시점'].str[5:7].astype(int)
coc[["건설_소계", "주거용건물", "비주거용건물", "건축보수"]] = coc[["건설_소계", "주거용건물", "비주거용건물", "건축보수"]].astype(float)
coc['거래분기'] = ((coc['월'] - 1) // 3 + 1).astype(str) + 'Q'
coc = coc.drop(["시점"], axis=1)

# 연도,분기별 건설공사비지수 평균 계산
df5_final = coc.groupby(['거래년도', '거래분기']).mean().reset_index()
df5_final.drop(columns=["월"], inplace=True)

# 변수명 변환
df5_final.rename(columns={
    "건설_소계": "건설_소계_건설공사비지수",
    "주거용건물": "주거용건물_건설공사비지수",
    "비주거용건물": "비주거용건물_건설공사비지수",
    "건축보수": "건축보수_건설공사비지수"
}, inplace=True)

display(df5_final.head())
display(df5_final.tail())

,거래년도,거래분기,건설_소계_건설공사비지수,주거용건물_건설공사비지수,비주거용건물_건설공사비지수,건축보수_건설공사비지수
0,2014,1Q,84.696667,83.910000,84.670000,84.730000
1,2014,2Q,84.813333,84.243333,84.850000,84.866667
2,2014,3Q,84.930000,84.373333,84.963333,85.140000
3,2014,4Q,85.420000,84.930000,85.453333,85.763333
4,2015,1Q,85.490000,85.106667,85.550000,85.983333


,거래년도,거래분기,건설_소계_건설공사비지수,주거용건물_건설공사비지수,비주거용건물_건설공사비지수,건축보수_건설공사비지수
37,2023,2Q,127.420000,126.466667,127.290000,126.130000
38,2023,3Q,128.020000,127.000000,127.823333,126.623333
39,2023,4Q,128.950000,128.003333,128.726667,127.576667
40,2024,1Q,129.956667,129.093333,129.676667,128.580000
41,2024,2Q,130.130000,129.043333,129.690000,128.700000


## 6. 표준지공시지가평균(원/㎡)
- 출처 : [공공데이터포털 - 국토교통부](https://www.data.go.kr/data/15004246/fileData.do)
- raw_data명 : '표준지공사지가/국토교통부_표준지공시지가_{year}.csv'
- raw_data형식 : 연도별, 필지별, 면적, 표준지공시지가
- 기간 : 2014 - 2024, 전국
- 형태변환 : 연도별, 분기별, 동별 표준지공시지가 평균 계산


### 6-1. 2016년 제외(2014-2024년)

In [21]:
# 2014년부터 2024년까지 for문으로 반복 처리 (2016년 제외)

for year in [2014, 2015, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024]:
    
    # 데이터 불러오기
    try:
        df = pd.read_csv(f'../data/raw/표준지공시지가/국토교통부_표준지공시지가_{year}.csv', encoding='utf-8')
    except:
        df = pd.read_csv(f'../data/raw/표준지공시지가/국토교통부_표준지공시지가_{year}.csv', encoding='cp949')
    
    # 필요한 열 선택
    df = df.loc[:, ['시도명', '시군구명', '소재지', '공시지가', '면적']]
    
    # '경기도', '서울특별시' 필터링
    df = df[df['시도명'].isin(['경기도', '서울특별시'])]
    
    # 면적당 공시지가 계산
    df['면적당공시지가'] = df['공시지가'] / df['면적']
    
    # '동' 추출
    df['동'] = df['소재지'].str.extract(r'(\S+[동가])')
    
    # '동'별로 면적당 공시지가 평균 계산
    df_cast = df.groupby('동', as_index=False)['면적당공시지가'].mean()
    
    # 거래년도 추가
    df_cast['거래년도'] = year
    
    # 열 이름 변경
    df_cast.rename(columns={'면적당공시지가': '표준지공시지가평균(원/㎡)'}, inplace=True)
    
    # 연도별로 각각 df에 저장
    globals()[f'df6_{year}'] = df_cast

### 6-2. 2016년
다른 연도의 데이터 파일과 변수명이 달라, 따로 전처리

In [22]:
# 데이터 불러오기
df2016 = pd.read_csv('../data/raw/표준지공시지가/국토교통부_표준지공시지가_2016.csv', encoding='cp949')

# 필요한 데이터만 추출
df2016 = df2016.loc[:,['시도','동리','공시지가','면적']]
df2016 = df2016[df2016['시도'].isin(['경기도','서울특별시'])]

# 면적당공시지가 계산
df2016['면적당공시지가'] = df2016['공시지가']/df2016['면적']
df2016['동'] = df2016['동리'].str.extract(r'(\S+[동가])')

# 동별 면적당 표준공시지가평균 계산
df6_2016 = df2016.groupby('동', as_index=False)['면적당공시지가'].mean()
df6_2016['거래년도'] = 2016
df6_2016.rename(columns={'면적당공시지가': '표준지공시지가평균(원/㎡)'}, inplace=True)

### 모든 표준지공시지가 데이터 병합

In [23]:
# 연도별 데이터프레임을 리스트로 모아 concat
df6_list = [globals()[f'df6_{year}'] for year in range(2014,2025)]
df6_final = pd.concat(df6_list, ignore_index=True)

display(df6_final.head())
display(df6_final.tail())

,동,표준지공시지가평균(원/㎡),거래년도
0,가능동,7901.121888,2014
1,가락동,20344.887129,2014
2,가리봉동,22912.129305,2014
3,가사동,600.462885,2014
4,가산동,11104.302339,2014


,동,표준지공시지가평균(원/㎡),거래년도
11000,후암동,53056.721449,2024
11001,훈정동,5.738515,2024
11002,휘경동,27719.473116,2024
11003,흑석동,51005.283730,2024
11004,흥인동,140455.230834,2024


## 7. 상업용 임대동향 지표(오피스_)
오피스_ 평균연면적(㎡) / 공실률(%) / 전환율(%) / 임대료(천원/㎡) /   
임대수입(%) / 기타수입(%) / 영업경비(%) / 순영업소득(%) / 순영업소득(천원/㎡)/  
소득수익률(%) / 자본수익률(%) / 투자수익률(%)
- 출처 : [부동산 통계정보 - 상업용부동산 임대동향조사](https://www.reb.or.kr/r-one/portal/bbs/rpt/searchBulletinPage.do;jsessionid=102E979CDD69D2BD35722687DEEAE605)
- raw_data명 : '24년 2분기 상업용부동산 임대동향조사 통계표(공표용).xlsx'
- raw_data형식 : 권역별, 연도별, 월별, 각 sheet별 변수(지표)
- 기간 : 2014.01-2024.02, 전국
- 형태변환 : 연도별, 분기별, 동별, 각 지표

raw_data

In [24]:
# sheet별로 지표 추출
오피스임대동향 = {}
for sheet_num in ['101','103','105','113','114','115']:
    df_tmp = pd.read_excel('../data/raw/24년 2분기 상업용부동산 임대동향조사 통계표(공표용).xlsx', sheet_name=sheet_num)
    오피스임대동향[df_tmp.iloc[1,0]] = df_tmp

오피스임대동향.keys()

dict_keys([' (1) 빌딩 기본정보', ' (2) 공실률 - ① 공실률(전체)', ' (3) 임대료(3층이상) - ① 임대료(전체)', ' (5) 전환율', ' (1) 투자수익률', ' (2) 순영업소득'])

형태 변환

In [25]:
# 데이터프레임 형태 변환
df7 = pd.DataFrame()

for key in 오피스임대동향.keys():    
    df_tmp = 오피스임대동향[key]

    # 3번째 행을 컬럼명으로 설정, 1~3행 삭제
    df_tmp.columns = df_tmp.iloc[2]
    df_tmp = df_tmp.drop([0,1,2],axis=0).reset_index(drop=True)
    
    #' (1) 투자수익률' 시트에만 '상세코드'와 '항목CODE'가 있어 오류나는 걸 해결
    try:
        df_tmp = df_tmp.drop(['지역CODE','상세코드','항목CODE'],axis=1)
    except:
        df_tmp = df_tmp.drop(['지역CODE'],axis=1)
    
    # 서울과 분당만 남김
    df_tmp = df_tmp[(df_tmp['지역구분(1)'] == '서울')|(df_tmp['지역구분(2)'] == '분당역세권')]
        
    df7 = pd.concat([df7, df_tmp],axis=0)

# 2014년 이후 데이터만 남김
df7.drop(['2013.1Q','2013.2Q','2013.3Q','2013.4Q'], axis=1, inplace=True)
df7 = df7.sort_values(by='항목').reset_index(drop=True)

In [26]:
# 지역 - 항목 - 기간 - 값 형태로 melt
df7_melt = pd.melt(df7, id_vars=['지역구분(1)','지역구분(2)','항목'], var_name='기간', value_name='값')

# 기간을 거래년도와 거래분기로 분리
df7_melt['거래년도'] = df7_melt['기간'].apply(lambda x: x[:4]).astype(int)
df7_melt['거래분기'] = df7_melt['기간'].apply(lambda x: x[5:])

# 컬럼 순서 재정렬
df7_melt = df7_melt[['거래년도','거래분기','지역구분(1)','지역구분(2)','항목','값']]

# 행 재정렬
df7_melt = df7_melt.sort_values(by=['거래년도','거래분기','지역구분(1)','지역구분(2)']).reset_index(drop=True)

print('기간당 데이터 종류 수:',len(df7_melt['항목'].unique()))
print('기간당 데이터 종류:', df7_melt['항목'].unique())
display(df7_melt.head(15))

# 지역 매칭 필요
print('**지역구분**\n',df7_melt['지역구분(2)'].unique())

# 변수명 설정을 위해 항목 앞에 '오피스_' 추가
df7_melt['항목'] = df7_melt['항목'].apply(lambda x: '오피스_'+x)

기간당 데이터 종류 수: 14
기간당 데이터 종류: ['공실률(%)' '기타수입(%)' '동수(동)' '소득수익률(%)' '순영업소득(%)' '순영업소득(천원/㎡)' '영업경비(%)'
 '임대료(천원/㎡)' '임대수입(%)' '자본수익률(%)' '전환율(%)' '투자수익률(%)' '평균연면적(㎡)' '평균층수(층)']


,거래년도,거래분기,지역구분(1),지역구분(2),항목,값
0,2014,1Q,경기,분당역세권,공실률(%),21.5
1,2014,1Q,경기,분당역세권,기타수입(%),0.577956
2,2014,1Q,경기,분당역세권,동수(동),6
3,2014,1Q,경기,분당역세권,소득수익률(%),1.692942
4,2014,1Q,경기,분당역세권,순영업소득(%),70.132352
5,2014,1Q,경기,분당역세권,순영업소득(천원/㎡),36
6,2014,1Q,경기,분당역세권,영업경비(%),29.867648
7,2014,1Q,경기,분당역세권,임대료(천원/㎡),16.516719
8,2014,1Q,경기,분당역세권,임대수입(%),99.422044
9,2014,1Q,경기,분당역세권,자본수익률(%),-0.329497


**지역구분**
 ['분당역세권' '강남대로' '계' '공덕역' '광화문' '교대역' '남대문' '남부터미널' '논현역' '당산역' '도산대로'
 '동대문' '명동' '목동' '사당' '소계(강남지역)' '소계(기타지역)' '소계(도심지역)' '소계(여의도마포지역)'
 '숙명여대' '시청' '신사역' '여의도' '영등포역' '용산역' '을지로' '잠실/송파' '잠실새내역' '장안동' '종로'
 '천호' '충무로' '테헤란로' '홍대/합정' '화곡']


In [27]:
# 항목과 값을 cast
df7_filtered = df7_melt.pivot_table(index=['거래년도', '거래분기', '지역구분(1)', '지역구분(2)'], columns='항목', values='값').reset_index()

# 사용할 열만 추출
df7_final = df7_filtered[['거래년도', '거래분기', '지역구분(2)', 
                       '오피스_평균연면적(㎡)','오피스_공실률(%)','오피스_전환율(%)','오피스_임대료(천원/㎡)',
                       '오피스_임대수입(%)','오피스_기타수입(%)','오피스_영업경비(%)','오피스_순영업소득(%)', '오피스_순영업소득(천원/㎡)',
                       '오피스_소득수익률(%)','오피스_자본수익률(%)', '오피스_투자수익률(%)']]
df7_final.rename(columns={'지역구분(2)': '권역2'}, inplace=True)
df7_final.columns.name = None
df7_final.head()

,거래년도,거래분기,권역2,오피스_평균연면적(㎡),오피스_공실률(%),오피스_전환율(%),오피스_임대료(천원/㎡),오피스_임대수입(%),오피스_기타수입(%),오피스_영업경비(%),오피스_순영업소득(%),오피스_순영업소득(천원/㎡),오피스_소득수익률(%),오피스_자본수익률(%),오피스_투자수익률(%)
0,2014,1Q,분당역세권,16729.0,21.5,12.0,16.516719,99.422044,0.577956,29.867648,70.132352,36.0,1.692942,-0.329497,1.363445
1,2014,1Q,강남대로,8970.0,12.8,12.4,23.854682,99.728242,0.271758,19.561652,80.438348,68.0,0.966286,0.248596,1.214881
2,2014,1Q,계,9266.0,8.9,11.8,20.441567,98.909512,1.090488,23.798965,76.201035,53.0,1.391067,0.184811,1.575878
3,2014,1Q,공덕역,20721.0,3.2,10.2,16.369239,99.144424,0.855576,31.969302,68.030698,46.0,1.566998,-0.053708,1.51329
4,2014,1Q,광화문,15512.0,7.8,11.5,21.307219,99.053264,0.946736,26.098656,73.901344,55.0,1.609872,0.052319,1.662191


## 8. 주거용_실거래가
주거용 / 단독다가구 / 연립다세대 / 아파트 / 오피스텔 _실거래가_건물면적당평균(만원/㎡)
- 출처 : [서울열린데이터광장 - 서울시 부동산 실거래가 정보](https://data.seoul.go.kr/dataList/OA-21275/S/1/datasetView.do) (계약일 기준 2014-2024)
- raw_data명 : '서울시 부동산 실거래가 정보/서울시 부동산 실거래가 정보 {year}.csv'
- raw_data형식 : 일별, 건물별, 실거래가, 건물용도
- 기간 : (계약일 기준) 2014.01.01-202410.02, 서울시
- 형태변환 : 연도별, 분기별, 동별, 실거래가평균

raw_data

In [28]:
# 2014-2024년 파일 병합

data_list = []
for year in range(2014, 2025):
    df = pd.read_csv( f"../data/raw/서울시 부동산 실거래가 정보/서울시 부동산 실거래가 정보 {year}.csv",encoding='cp949')
    data_list.append(df)

combined_data = pd.concat(data_list, ignore_index=True)

# 데이터프레임 형태로 변환
df8 = pd.DataFrame(combined_data)

print('기간 :',df8.계약일.min(), df8.계약일.max())
df8.head()

기간 : 20140101 20241002


,접수연도,자치구코드,자치구명,법정동코드,법정동명,지번구분,지번구분명,본번,부번,건물명,계약일,물건금액(만원),건물면적(㎡),토지면적(㎡),층,권리구분,취소일,건축년도,건물용도,신고구분,신고한 개업공인중개사 시군구명
0,2015,11560,영등포구,13300,대림동,1.0,대지,1112.0,4.0,(1112-4),20141231,12000,38.80,16.14,1.0,NaN,NaN,1994.0,연립다세대,NaN,NaN
1,2015,11650,서초구,10100,방배동,NaN,NaN,NaN,NaN,NaN,20141231,162000,322.80,186.80,NaN,NaN,NaN,1993.0,단독다가구,NaN,NaN
2,2015,11170,용산구,13600,보광동,1.0,대지,3.0,10.0,(3-10),20141231,36000,47.39,22.25,1.0,NaN,NaN,1993.0,연립다세대,NaN,NaN
3,2015,11590,동작구,10700,사당동,NaN,NaN,NaN,NaN,NaN,20141231,180000,517.13,165.00,NaN,NaN,NaN,1996.0,단독다가구,NaN,NaN
4,2015,11410,서대문구,11700,연희동,1.0,대지,724.0,16.0,(724-16),20141231,4000,36.48,14.10,-1.0,NaN,NaN,1989.0,연립다세대,NaN,NaN


형태 변환

In [29]:
# 계약일 컬럼을 datetime 형식으로 변환
df8['계약일'] = pd.to_datetime(df8['계약일'], format='%Y%m%d')

# 연도와 분기 추출하여 새로운 컬럼으로 추가
df8['거래년도'] = df8['계약일'].dt.year
df8['거래분기'] = df8['계약일'].dt.quarter
df8['거래분기'] = df8['거래분기'].apply(lambda x: str(x)+"Q")

# 건물면적당 거래가 생성
df8['주거용_실거래가_건물면적당평균(만원/㎡)'] = df8['물건금액(만원)'] / df8['건물면적(㎡)']

# 데이터 형태 변경
df8.rename(columns ={'법정동명':'동','자치구명':'구'},inplace=True)

평균거래가_종합 = df8[['거래년도','거래분기','구','동','주거용_실거래가_건물면적당평균(만원/㎡)']].groupby(['거래년도','거래분기','구','동']).mean().reset_index()
display(평균거래가_종합.head())
display(평균거래가_종합.tail())

평균거래가_건물용도별 = df8[['거래년도','거래분기','구','동','건물용도','주거용_실거래가_건물면적당평균(만원/㎡)']].groupby(['거래년도','거래분기','구','동','건물용도']).mean().reset_index()
display(평균거래가_건물용도별.head())
display(평균거래가_건물용도별.tail())

,거래년도,거래분기,구,동,주거용_실거래가_건물면적당평균(만원/㎡)
0,2014,1Q,강남구,개포동,1143.856825
1,2014,1Q,강남구,논현동,720.913872
2,2014,1Q,강남구,대치동,1014.114945
3,2014,1Q,강남구,도곡동,978.393028
4,2014,1Q,강남구,삼성동,952.716791


,거래년도,거래분기,구,동,주거용_실거래가_건물면적당평균(만원/㎡)
15962,2024,4Q,영등포구,문래동5가,424.219051
15963,2024,4Q,영등포구,양평동1가,1476.087384
15964,2024,4Q,영등포구,양평동2가,606.388088
15965,2024,4Q,영등포구,여의도동,1581.893405
15966,2024,4Q,중랑구,면목동,915.873880


,거래년도,거래분기,구,동,건물용도,주거용_실거래가_건물면적당평균(만원/㎡)
0,2014,1Q,강남구,개포동,아파트,1273.373395
1,2014,1Q,강남구,개포동,연립다세대,505.744845
2,2014,1Q,강남구,개포동,오피스텔,458.774941
3,2014,1Q,강남구,논현동,단독다가구,713.731962
4,2014,1Q,강남구,논현동,아파트,799.790201


,거래년도,거래분기,구,동,건물용도,주거용_실거래가_건물면적당평균(만원/㎡)
43220,2024,4Q,영등포구,문래동5가,오피스텔,424.219051
43221,2024,4Q,영등포구,양평동1가,아파트,1476.087384
43222,2024,4Q,영등포구,양평동2가,오피스텔,606.388088
43223,2024,4Q,영등포구,여의도동,아파트,1581.893405
43224,2024,4Q,중랑구,면목동,아파트,915.873880


In [30]:
평균거래가_종합.to_csv('../data/processed1_주거용_실거래가_연면적당평균.csv', index=False, encoding='cp949')
평균거래가_건물용도별.to_csv('../data/processed1_주거용_실거래가_연면적당평균_건물용도별.csv', index=False, encoding='cp949')

In [31]:
평균거래가_건물용도별['건물용도'] = 평균거래가_건물용도별['건물용도'] + '_실거래가_건물면적당평균(만원/㎡)'
df8_final = 평균거래가_건물용도별.pivot_table(index=['거래년도', '거래분기','구','동'], columns='건물용도', values='주거용_실거래가_건물면적당평균(만원/㎡)').reset_index()
df8_final = pd.merge(df8_final, 평균거래가_종합, on=['거래년도','거래분기','구','동'])
                     
display(df8_final.head())
display(df8_final.tail())

,거래년도,거래분기,구,동,단독다가구_실거래가_건물면적당평균(만원/㎡),아파트_실거래가_건물면적당평균(만원/㎡),연립다세대_실거래가_건물면적당평균(만원/㎡),오피스텔_실거래가_건물면적당평균(만원/㎡),주거용_실거래가_건물면적당평균(만원/㎡)
0,2014,1Q,강남구,개포동,NaN,1273.373395,505.744845,458.774941,1143.856825
1,2014,1Q,강남구,논현동,713.731962,799.790201,614.019391,663.488648,720.913872
2,2014,1Q,강남구,대치동,474.600681,1120.629834,569.903933,706.500709,1014.114945
3,2014,1Q,강남구,도곡동,NaN,985.862771,594.959146,859.427572,978.393028
4,2014,1Q,강남구,삼성동,1546.901086,974.634463,673.838904,709.355759,952.716791


,거래년도,거래분기,구,동,단독다가구_실거래가_건물면적당평균(만원/㎡),아파트_실거래가_건물면적당평균(만원/㎡),연립다세대_실거래가_건물면적당평균(만원/㎡),오피스텔_실거래가_건물면적당평균(만원/㎡),주거용_실거래가_건물면적당평균(만원/㎡)
15962,2024,4Q,영등포구,문래동5가,NaN,NaN,NaN,424.219051,424.219051
15963,2024,4Q,영등포구,양평동1가,NaN,1476.087384,NaN,NaN,1476.087384
15964,2024,4Q,영등포구,양평동2가,NaN,NaN,NaN,606.388088,606.388088
15965,2024,4Q,영등포구,여의도동,NaN,1581.893405,NaN,NaN,1581.893405
15966,2024,4Q,중랑구,면목동,NaN,915.873880,NaN,NaN,915.873880


## 9. 오피스매매가격지수
- 출처 : [KB부동산 데이터 허브 - 권역분류기준](https://data.kbland.kr/kbstats/kbOffice)
- raw_data명 : '분기별 KB오피스 투자지수 매매가격지수.xlsx'
- 형태변환 : 권역별, 연도, 분기별 오피스매매가격지수

raw_data

In [32]:
df9 = pd.read_excel('../data/raw/분기별 KB오피스 투자지수 매매가격지수.xlsx')
df9.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 100 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   권역       8 non-null      object 
 1   구분       8 non-null      object 
 2   2000.1Q  8 non-null      float64
 3   2000.2Q  8 non-null      float64
 4   2000.3Q  8 non-null      float64
 5   2000.4Q  8 non-null      float64
 6   2001.1Q  8 non-null      float64
 7   2001.2Q  8 non-null      float64
 8   2001.3Q  8 non-null      float64
 9   2001.4Q  8 non-null      float64
 10  2002.1Q  8 non-null      float64
 11  2002.2Q  8 non-null      float64
 12  2002.3Q  8 non-null      float64
 13  2002.4Q  8 non-null      float64
 14  2003.1Q  8 non-null      float64
 15  2003.2Q  8 non-null      float64
 16  2003.3Q  8 non-null      float64
 17  2003.4Q  8 non-null      float64
 18  2004.1Q  8 non-null      float64
 19  2004.2Q  8 non-null      float64
 20  2004.3Q  8 non-null      float64
 21  2004.4Q  8 non-null

형태 변환

In [33]:
# '지수'라는 값밖에 없는 '구분'열 삭제
df9.drop('구분', axis=1,inplace=True)

# 권역 - 기간 - 가격지수 형태로 melt
df9_final = pd.melt(df9, id_vars='권역', var_name='기간', value_name='오피스매매가격지수')

# 기간을 거래년도, 거래분기로 분리
df9_final['거래년도'] = df9_final['기간'].apply(lambda x: x[:4]).astype(int)
df9_final['거래분기'] = df9_final['기간'].apply(lambda x: x[5:])

# 2014년 이후 데이터만 추출, 열 순서 재정렬
df9_final = df9_final.loc[df9_final['거래년도']>=2014,:].reset_index(drop=True)[['거래년도','거래분기','권역','오피스매매가격지수']]
display(df9_final.head())
display(df9_final.tail())

print('**권역구분**')
df9_final['권역'].unique()

,거래년도,거래분기,권역,오피스매매가격지수
0,2014,1Q,서울,54.725681
1,2014,1Q,도심권,57.913920
2,2014,1Q,강남권,50.959516
3,2014,1Q,여의도권,55.544697
4,2014,1Q,기타권,55.021148


,거래년도,거래분기,권역,오피스매매가격지수
331,2024,2Q,여의도권,103.182304
332,2024,2Q,기타권,101.961123
333,2024,2Q,영등포구로권,101.944307
334,2024,2Q,강동송파권,102.206767
335,2024,2Q,기타,101.893079


**권역구분**


array(['서울', '도심권', '강남권', '여의도권', '기타권', '영등포구로권', '강동송파권', '기타'],
      dtype=object)

## 10. 권역
- 출처 : [KB부동산 데이터 허브 - 권역분류기준](https://data.kbland.kr/kbstats/kbOffice)

1. 도심권(CBD): 종로구, 중구, 용산구 동자동  
2. 강남권(GBD:) 강남구, 서초구  
3. 여의도권(YBD): 영등포구 여의도동, 마포구 마포동, 도화동, 공덕동  
4. 기타권(OBD): 도심권, 강남권, 여의도권 외 서울 전 지역  
5. 영등포구로권(YGD): 영등포구 여의도동 외 전지역, 구로구 전체  
6. 강동송파권(GSD): 강동구, 송파구  
7. 기타(Others): 도심권, 강남권, 여의도권, 영등포구로권, 강동송파권을 제외한 서울지역 + 분당구  
*분당이 분류기준상 없지만, 분당도 기타지역으로 추가

In [34]:
# 권역 분류 함수 정의
def assign_area(row):
    gu = row['구']
    dong = row['동']
    
    if gu in ['종로구', '중구', '용산구'] and dong == '동자동':
        return '도심권'
    elif gu in ['강남구', '서초구']:
        return '강남권'
    elif (gu == '영등포구' and dong == '여의도동') or (gu == '마포구' and dong in ['마포동', '도화동', '공덕동']):
        return '여의도권'
    elif gu == '영등포구' and dong != '여의도동':
        return '영등포구로권'
    elif gu == '구로구':
        return '영등포구로권'
    elif gu in ['강동구', '송파구']:
        return '강동송파권'
    else:
        return '기타권'

# 새로운 열 '권역' 추가
df0['권역'] = df0.apply(assign_area, axis=1)
df0['권역'].value_counts(dropna=False)

권역
기타권       442
강남권       357
여의도권       58
강동송파권      58
영등포구로권     54
도심권         3
Name: count, dtype: int64

# merge
제공데이터 기준으로 병합

### 제공데이터

In [35]:
df0.head()

,거래년도,거래분기,시,구,동,거래당시연면적,거래당시토지면적,현재연면적,빌딩비고,거래비고,거래가,지상층,지하층,준공년도,건물유형,권역
0,2014,1Q,서울,영등포구,여의도동,27202.05,3602.0,NaN,NaN,NaN,14600,12.0,4.0,1985.0,집합,여의도권
1,2014,1Q,서울,강남구,논현동,3516.43,713.6,NaN,NaN,NaN,8100,7.0,2.0,2004.0,일반,강남권
2,2014,1Q,서울,서대문구,충정로3가,25984.49,2705.0,NaN,NaN,NaN,13900,17.0,4.0,1995.0,일반,기타권
3,2014,1Q,서울,서대문구,충정로3가,31894.78,3772.0,NaN,NaN,NaN,10500,17.0,4.0,1987.0,집합,기타권
4,2014,1Q,서울,용산구,한남동,4370.31,730.5,NaN,NaN,NaN,25200,5.0,4.0,2012.0,일반,기타권


### 전처리 데이터

In [36]:
# APT매매가격지수 - 거래년도, 거래분기, 동
df1_final.head()

,구,거래년도,거래분기,APT매매가격지수
0,강남구,2014,3Q,55.878059
1,강남구,2014,4Q,56.321412
2,강남구,2015,1Q,56.618775
3,강남구,2015,2Q,57.522446
4,강남구,2015,3Q,58.733925


In [37]:
# 업무용_실거래가_연면적당평균(만원/㎡) - 거래년도, 거래분기, 시, 구, 동
df2_final.head()

,거래년도,거래분기,시,구,동,업무용_실거래가_연면적당평균(만원/㎡)
0,2014,1Q,성남,분당구,구미동,354.012138
1,2014,1Q,성남,분당구,궁내동,268.600591
2,2014,1Q,성남,분당구,금곡동,268.101328
3,2014,1Q,성남,분당구,삼평동,436.963754
4,2014,1Q,성남,분당구,서현동,610.069339


In [38]:
# 기준금리(%) - 거래년도, 거래분기
df3_final.head()

,기준금리(%),거래년도,거래분기
0,2.50,2014,1Q
1,2.50,2014,2Q
2,2.00,2014,3Q
3,2.00,2014,4Q
4,1.75,2015,1Q


In [39]:
# 소비자물가지수 - 거래년도, 거래분기, 시
df4_final.head()

,시,소비자물가지수,거래년도,거래분기
0,서울,92.885,2014,1Q
1,서울,93.334,2014,2Q
2,서울,93.638,2014,3Q
3,서울,93.343,2014,4Q
4,서울,94.005,2015,1Q


In [40]:
# 건설비공사지수 - 거래년도, 거래분기
df5_final.head()

,거래년도,거래분기,건설_소계_건설공사비지수,주거용건물_건설공사비지수,비주거용건물_건설공사비지수,건축보수_건설공사비지수
0,2014,1Q,84.696667,83.910000,84.670000,84.730000
1,2014,2Q,84.813333,84.243333,84.850000,84.866667
2,2014,3Q,84.930000,84.373333,84.963333,85.140000
3,2014,4Q,85.420000,84.930000,85.453333,85.763333
4,2015,1Q,85.490000,85.106667,85.550000,85.983333


In [41]:
# 표준지공시지가평균(원/㎡) - 거래년도, 동
df6_final.head()

,동,표준지공시지가평균(원/㎡),거래년도
0,가능동,7901.121888,2014
1,가락동,20344.887129,2014
2,가리봉동,22912.129305,2014
3,가사동,600.462885,2014
4,가산동,11104.302339,2014


In [42]:
# 상업용 임대동향 지표(오피스_) - 거래년도, 거래분기, 권역2
df7_final.head()

,거래년도,거래분기,권역2,오피스_평균연면적(㎡),오피스_공실률(%),오피스_전환율(%),오피스_임대료(천원/㎡),오피스_임대수입(%),오피스_기타수입(%),오피스_영업경비(%),오피스_순영업소득(%),오피스_순영업소득(천원/㎡),오피스_소득수익률(%),오피스_자본수익률(%),오피스_투자수익률(%)
0,2014,1Q,분당역세권,16729.0,21.5,12.0,16.516719,99.422044,0.577956,29.867648,70.132352,36.0,1.692942,-0.329497,1.363445
1,2014,1Q,강남대로,8970.0,12.8,12.4,23.854682,99.728242,0.271758,19.561652,80.438348,68.0,0.966286,0.248596,1.214881
2,2014,1Q,계,9266.0,8.9,11.8,20.441567,98.909512,1.090488,23.798965,76.201035,53.0,1.391067,0.184811,1.575878
3,2014,1Q,공덕역,20721.0,3.2,10.2,16.369239,99.144424,0.855576,31.969302,68.030698,46.0,1.566998,-0.053708,1.51329
4,2014,1Q,광화문,15512.0,7.8,11.5,21.307219,99.053264,0.946736,26.098656,73.901344,55.0,1.609872,0.052319,1.662191


In [43]:
# 표준지공시지가평균(원/㎡) - 거래년도, 거래분기, 구, 동
df8_final.head()

,거래년도,거래분기,구,동,단독다가구_실거래가_건물면적당평균(만원/㎡),아파트_실거래가_건물면적당평균(만원/㎡),연립다세대_실거래가_건물면적당평균(만원/㎡),오피스텔_실거래가_건물면적당평균(만원/㎡),주거용_실거래가_건물면적당평균(만원/㎡)
0,2014,1Q,강남구,개포동,NaN,1273.373395,505.744845,458.774941,1143.856825
1,2014,1Q,강남구,논현동,713.731962,799.790201,614.019391,663.488648,720.913872
2,2014,1Q,강남구,대치동,474.600681,1120.629834,569.903933,706.500709,1014.114945
3,2014,1Q,강남구,도곡동,NaN,985.862771,594.959146,859.427572,978.393028
4,2014,1Q,강남구,삼성동,1546.901086,974.634463,673.838904,709.355759,952.716791


In [44]:
# 표준지공시지가평균(원/㎡) - 거래년도, 거래분기, 권역
df9_final.head()

,거래년도,거래분기,권역,오피스매매가격지수
0,2014,1Q,서울,54.725681
1,2014,1Q,도심권,57.913920
2,2014,1Q,강남권,50.959516
3,2014,1Q,여의도권,55.544697
4,2014,1Q,기타권,55.021148


### MERGE

In [45]:
df0.head()

,거래년도,거래분기,시,구,동,거래당시연면적,거래당시토지면적,현재연면적,빌딩비고,거래비고,거래가,지상층,지하층,준공년도,건물유형,권역
0,2014,1Q,서울,영등포구,여의도동,27202.05,3602.0,NaN,NaN,NaN,14600,12.0,4.0,1985.0,집합,여의도권
1,2014,1Q,서울,강남구,논현동,3516.43,713.6,NaN,NaN,NaN,8100,7.0,2.0,2004.0,일반,강남권
2,2014,1Q,서울,서대문구,충정로3가,25984.49,2705.0,NaN,NaN,NaN,13900,17.0,4.0,1995.0,일반,기타권
3,2014,1Q,서울,서대문구,충정로3가,31894.78,3772.0,NaN,NaN,NaN,10500,17.0,4.0,1987.0,집합,기타권
4,2014,1Q,서울,용산구,한남동,4370.31,730.5,NaN,NaN,NaN,25200,5.0,4.0,2012.0,일반,기타권


In [46]:
# 기준금리(%) - 거래년도, 거래분기
df_result = pd.merge(df0, df3_final, on=['거래년도', '거래분기'], how='left')

# APT매매가격지수 - 구, 거래년도, 거래분기
df_result = pd.merge(df_result, df1_final, on=['구','거래년도','거래분기'], how='left') 

# 오피스매매가격지수 - 거래년도, 거래분기, 권역
df_result = pd.merge(df_result, df9_final, on=['권역','거래년도','거래분기'], how='left')

# 소비자물가지수 - 거래년도, 거래분기, 시
df_result = pd.merge(df_result, df4_final, on=['시','거래년도','거래분기'], how='left') 

# 건설비공사지수(건설_소계, 주거용건물, 비주거용건물, 건축보수) - 거래년도, 거래분기
df_result = pd.merge(df_result, df5_final, on=['거래년도','거래분기'], how='left')  

# 표준지공시지가평균(원/㎡) - 거래년도, 동
df_result = pd.merge(df_result, df6_final, on=['동','거래년도'], how='left')

# 업무용_실거래가_연면적당평균(만원/㎡) - 거래년도, 거래분기, 시, 구, 동
df_result = pd.merge(df_result, df2_final, on=['시','구','동','거래년도','거래분기'], how='left') 

# 주거용_실거래가 - 거래년도, 거래분기, 구, 동
df_result = pd.merge(df_result, df8_final, on=['동','구','거래년도','거래분기'], how='left')

display(df_result.iloc[:,15:].info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972 entries, 0 to 971
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   권역                        972 non-null    object 
 1   기준금리(%)                   972 non-null    object 
 2   APT매매가격지수                 938 non-null    object 
 3   오피스매매가격지수                 972 non-null    float64
 4   소비자물가지수                   972 non-null    float64
 5   건설_소계_건설공사비지수             972 non-null    float64
 6   주거용건물_건설공사비지수             972 non-null    float64
 7   비주거용건물_건설공사비지수            972 non-null    float64
 8   건축보수_건설공사비지수              972 non-null    float64
 9   표준지공시지가평균(원/㎡)            972 non-null    float64
 10  업무용_실거래가_연면적당평균(만원/㎡)     749 non-null    float64
 11  단독다가구_실거래가_건물면적당평균(만원/㎡)  636 non-null    float64
 12  아파트_실거래가_건물면적당평균(만원/㎡)    731 non-null    float64
 13  연립다세대_실거래가_건물면적당평균(만원/㎡)  638 non-null    float64
 14  오피스텔_실거래가_

None

In [47]:
# 상업용 임대동향 지표(오피스_) : 권역2, 거래년도, 거래분기


# 권역2 분류기준
categories = {
    '분당역세권': ['야탑동', '수내동', '서현동', '정자동'],
    '강남대로': ['논현동', '신사동', '삼성동', '청담동', '역삼동', '대치동', '서초동', '양재동'],
    '공덕역': ['공덕동', '도화동'],
    '광화문': ['서린동', '청진동', '종로2가', '종로6가'],
    '교대역': ['서초동'],
    '남대문': ['남대문로1가', '남대문로2가', '남대문로5가'],
    '남부터미널': ['서초동', '방배동'],
    '논현역': ['논현동'],
    '당산역': ['당산동1가', '당산동3가', '당산동4가', '당산동5가', '당산동6가'],
    '도산대로': ['신사동', '청담동'],
    '동대문': ['제기동', '휘경동', '청량리동'],
    '명동': ['명동1가', '필동1가', '필동2가', '필동3가'],
    '목동': ['목동'],
    '사당': ['사당동'],
    '숙명여대': ['남영동'],
    '시청': ['무교동', '태평로2가'],
    '신사역': ['신사동'],
    '여의도': ['여의도동'],
    '영등포역': ['영등포동2가', '영등포동4가', '영등포동6가', '영등포동7가'],
    '용산역': ['한강로2가', '한강로3가', '후암동'],
    '을지로': ['을지로1가', '을지로2가', '을지로3가', '을지로4가', '을지로6가'],
    '잠실/송파': ['잠실동', '신천동', '석촌동', '풍납동', '송파동', '오금동'],
    '잠실새내역': ['신천동'],
    '장안동': ['장안동'],
    '종로': ['서린동', '청진동', '종로2가', '종로6가'],
    '천호': ['천호동'],
    '충무로': ['충무로1가', '충무로3가'],
    '테헤란로': ['삼성동', '역삼동'],
    '홍대/합정': ['서교동', '합정동', '동교동'],
    '화곡': ['화곡동']
}

# 각 동을 권역2 기준으로 재분류
def classify_dong(dong):
    for category, dongs in categories.items():
        if dong in dongs:
            return category
    return '소계(기타지역)'

# 새로운 열 '권역2' 추가
df_result['권역2'] = df_result['동'].apply(classify_dong)
df_result['권역2'].value_counts(dropna=False)
print("권역2 추가 후 shape:",df_result.shape)

# 상업용 임대동향 지표(오피스_) merge - 거래년도, 거래분기, 권역2
df_final = pd.merge(df_result, df7_final, on=['거래년도','거래분기','권역2'], how='left')
df_final.drop(columns='권역2', inplace=True)
print("권역2 제거 후 shape:",df_final.shape)
print(df_final.columns)
print(df_final.info())
display(df_final.head())
display(df_final.tail())


권역2 추가 후 shape: (972, 32)
권역2 제거 후 shape: (972, 43)
Index(['거래년도', '거래분기', '시', '구', '동', '거래당시연면적', '거래당시토지면적', '현재연면적', '빌딩비고',
       '거래비고', '거래가', '지상층', '지하층', '준공년도', '건물유형', '권역', '기준금리(%)',
       'APT매매가격지수', '오피스매매가격지수', '소비자물가지수', '건설_소계_건설공사비지수', '주거용건물_건설공사비지수',
       '비주거용건물_건설공사비지수', '건축보수_건설공사비지수', '표준지공시지가평균(원/㎡)',
       '업무용_실거래가_연면적당평균(만원/㎡)', '단독다가구_실거래가_건물면적당평균(만원/㎡)',
       '아파트_실거래가_건물면적당평균(만원/㎡)', '연립다세대_실거래가_건물면적당평균(만원/㎡)',
       '오피스텔_실거래가_건물면적당평균(만원/㎡)', '주거용_실거래가_건물면적당평균(만원/㎡)', '오피스_평균연면적(㎡)',
       '오피스_공실률(%)', '오피스_전환율(%)', '오피스_임대료(천원/㎡)', '오피스_임대수입(%)',
       '오피스_기타수입(%)', '오피스_영업경비(%)', '오피스_순영업소득(%)', '오피스_순영업소득(천원/㎡)',
       '오피스_소득수익률(%)', '오피스_자본수익률(%)', '오피스_투자수익률(%)'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972 entries, 0 to 971
Data columns (total 43 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   거래년도                      972 

,거래년도,거래분기,시,구,동,거래당시연면적,거래당시토지면적,현재연면적,빌딩비고,거래비고,거래가,지상층,지하층,준공년도,건물유형,권역,기준금리(%),APT매매가격지수,오피스매매가격지수,소비자물가지수,건설_소계_건설공사비지수,주거용건물_건설공사비지수,비주거용건물_건설공사비지수,건축보수_건설공사비지수,표준지공시지가평균(원/㎡),업무용_실거래가_연면적당평균(만원/㎡),단독다가구_실거래가_건물면적당평균(만원/㎡),아파트_실거래가_건물면적당평균(만원/㎡),연립다세대_실거래가_건물면적당평균(만원/㎡),오피스텔_실거래가_건물면적당평균(만원/㎡),주거용_실거래가_건물면적당평균(만원/㎡),오피스_평균연면적(㎡),오피스_공실률(%),오피스_전환율(%),오피스_임대료(천원/㎡),오피스_임대수입(%),오피스_기타수입(%),오피스_영업경비(%),오피스_순영업소득(%),오피스_순영업소득(천원/㎡),오피스_소득수익률(%),오피스_자본수익률(%),오피스_투자수익률(%)
0,2014,1Q,서울,영등포구,여의도동,27202.05,3602.0,NaN,NaN,NaN,14600,12.0,4.0,1985.0,집합,여의도권,2.50,NaN,55.544697,92.885,84.696667,83.91,84.67,84.73,4097.330378,419.924977,NaN,724.699407,NaN,532.679833,680.294881,23158.0,8.8,12.0,20.607946,98.591924,1.408076,25.107995,74.892005,54.0,1.496556,0.275278,1.771833
1,2014,1Q,서울,강남구,논현동,3516.43,713.6,NaN,NaN,NaN,8100,7.0,2.0,2004.0,일반,강남권,2.50,NaN,50.959516,92.885,84.696667,83.91,84.67,84.73,22213.120943,NaN,713.731962,799.790201,614.019391,663.488648,720.913872,8970.0,12.8,12.4,23.854682,99.728242,0.271758,19.561652,80.438348,68.0,0.966286,0.248596,1.214881
2,2014,1Q,서울,서대문구,충정로3가,25984.49,2705.0,NaN,NaN,NaN,13900,17.0,4.0,1995.0,일반,기타권,2.50,NaN,55.021148,92.885,84.696667,83.91,84.67,84.73,32572.694881,NaN,NaN,511.106102,454.674188,477.732794,494.933719,6012.0,10.2,11.2,14.012983,98.790338,1.209662,27.511143,72.488857,36.0,1.237202,0.543683,1.780884
3,2014,1Q,서울,서대문구,충정로3가,31894.78,3772.0,NaN,NaN,NaN,10500,17.0,4.0,1987.0,집합,기타권,2.50,NaN,55.021148,92.885,84.696667,83.91,84.67,84.73,32572.694881,NaN,NaN,511.106102,454.674188,477.732794,494.933719,6012.0,10.2,11.2,14.012983,98.790338,1.209662,27.511143,72.488857,36.0,1.237202,0.543683,1.780884
4,2014,1Q,서울,용산구,한남동,4370.31,730.5,NaN,NaN,NaN,25200,5.0,4.0,2012.0,일반,기타권,2.50,NaN,55.021148,92.885,84.696667,83.91,84.67,84.73,23992.170118,NaN,744.626378,1090.179995,758.559705,793.650794,979.598932,6012.0,10.2,11.2,14.012983,98.790338,1.209662,27.511143,72.488857,36.0,1.237202,0.543683,1.780884


,거래년도,거래분기,시,구,동,거래당시연면적,거래당시토지면적,현재연면적,빌딩비고,거래비고,거래가,지상층,지하층,준공년도,건물유형,권역,기준금리(%),APT매매가격지수,오피스매매가격지수,소비자물가지수,건설_소계_건설공사비지수,주거용건물_건설공사비지수,비주거용건물_건설공사비지수,건축보수_건설공사비지수,표준지공시지가평균(원/㎡),업무용_실거래가_연면적당평균(만원/㎡),단독다가구_실거래가_건물면적당평균(만원/㎡),아파트_실거래가_건물면적당평균(만원/㎡),연립다세대_실거래가_건물면적당평균(만원/㎡),오피스텔_실거래가_건물면적당평균(만원/㎡),주거용_실거래가_건물면적당평균(만원/㎡),오피스_평균연면적(㎡),오피스_공실률(%),오피스_전환율(%),오피스_임대료(천원/㎡),오피스_임대수입(%),오피스_기타수입(%),오피스_영업경비(%),오피스_순영업소득(%),오피스_순영업소득(천원/㎡),오피스_소득수익률(%),오피스_자본수익률(%),오피스_투자수익률(%)
967,2024,2Q,서울,서초구,서초동,29784.83,4588.60,NaN,NaN,NaN,22500,14.0,5.0,1990.0,일반,강남권,3.50,96.355817,101.007436,113.32,130.13,129.043333,129.69,128.7,42315.651531,932.250811,943.637360,2577.502357,1229.101160,951.826774,1980.716163,11827.024264,9.903127,8.785714,28.870845,99.796874,0.203126,13.284225,86.715775,84.101704,0.821505,2.313352,3.134857
968,2024,2Q,서울,구로구,구로동,3556.72,819.70,NaN,NaN,NaN,12100,7.0,2.0,1990.0,일반,영등포구로권,3.50,87.266501,101.944307,113.32,130.13,129.043333,129.69,128.7,33337.713799,310.419288,402.366416,923.598932,800.187427,551.722568,845.662296,5613.686714,5.626018,7.769286,13.19913,98.431393,1.568607,24.546488,75.453512,39.970046,0.850216,0.407892,1.258107
969,2024,2Q,서울,영등포구,대림동,5464.08,1216.00,NaN,NaN,NaN,13300,10.0,2.0,1991.0,일반,영등포구로권,3.50,93.494579,101.944307,113.32,130.13,129.043333,129.69,128.7,26763.324530,NaN,515.173578,1091.962677,641.216749,NaN,902.726075,5613.686714,5.626018,7.769286,13.19913,98.431393,1.568607,24.546488,75.453512,39.970046,0.850216,0.407892,1.258107
970,2024,2Q,서울,광진구,자양동,4104.82,1541.40,NaN,NaN,NaN,14900,5.0,1.0,1990.0,일반,기타권,3.50,93.925438,101.961123,113.32,130.13,129.043333,129.69,128.7,34846.450652,NaN,1147.189061,1563.983856,1313.816038,1399.228517,1401.634106,5613.686714,5.626018,7.769286,13.19913,98.431393,1.568607,24.546488,75.453512,39.970046,0.850216,0.407892,1.258107
971,2024,2Q,서울,동대문구,신설동,7247.65,804.51,NaN,NaN,NaN,19600,14.0,3.0,2008.0,일반,기타권,3.50,86.398296,101.961123,113.32,130.13,129.043333,129.69,128.7,28296.555528,913.908290,NaN,956.343334,1435.678158,857.147025,1031.261256,5613.686714,5.626018,7.769286,13.19913,98.431393,1.568607,24.546488,75.453512,39.970046,0.850216,0.407892,1.258107


In [48]:
# csv로 저장
df_final.to_csv('../data/processed1_merged.csv',index=False, encoding='cp949')